In [1]:
import os
import csv
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from webdriver_manager.chrome import ChromeDriverManager
from selenium.common.exceptions import TimeoutException
import time
import requests
import pandas as pd

api_key = "3mM44UdC76NLG7_BRVpu5duXpY6Quz3NcKy71"
secret_key = "LvZyPvCGiboWySpNrHmD1z"

headers = {
    "Authorization": f"sso-key {api_key}:{secret_key}",
    "Content-Type": "application/json"
}

def check_domain_availability(domain_name):
    url = f"https://api.ote-godaddy.com/v1/domains/available?domain={domain_name}"
    try:
        response = requests.get(url, headers=headers)
        response.raise_for_status()  # Raise an error for bad responses
        
        data = response.json()
        if data.get('available'):
            print(f"{domain_name} is available!")
        else:
            print(f"{domain_name} is not available.")
    
    except requests.exceptions.RequestException as e:
        print(f"An error occurred: {e}")

def fetch_and_save_web_content(domains, output_csv_path, driver):
    """
    Fetch the content of each domain and save the results to a CSV file.

    :param domains: List of domains to fetch.
    :param output_csv_path: Path to the output CSV file.
    :param driver: A Selenium WebDriver instance.
    """
    # Prepare to write results to the output CSV file
    with open(output_csv_path, 'w', newline='', encoding='utf-8') as output_file:
        csv_writer = csv.writer(output_file)
        # Write headers to the output CSV
        csv_writer.writerow(['Domain', 'Content'])

        # Iterate over each domain and fetch the webpage content
        for domain in domains:
            try:
                url = f"{domain}"
                print(f"Fetching {url} ...")
                driver.get(url)
                
                # Extract the text content from the webpage body
                page_text = driver.find_element(By.TAG_NAME, "body").text

                # Write the domain and page text to the CSV file
                csv_writer.writerow([domain, page_text])
                print(f"Saved content for {domain}")

                # Pause for a second to avoid overloading the server (optional)
                time.sleep(1)

            except Exception as e:
                print(f"Failed to fetch {domain}: {e}")


def fetch_and_save_web_content_from_csv(input_csv_path, output_csv_path, driver, start_row=0, timeout=8):
    """
    Open the modified CSV, fetch web content for each domain starting from a specific row, and save results to a new CSV file.
    
    :param input_csv_path: Path to the input CSV file.
    :param output_csv_path: Path to the output CSV file.
    :param driver: A Selenium WebDriver instance.
    :param start_row: The row index to start processing from (default is 0).
    :param timeout: Time in seconds to wait for the page to load before considering it failed.
    """
    # Load the CSV file, skipping rows before start_row
    df = pd.read_csv(input_csv_path, skiprows=range(1, start_row + 1))  # Skips rows up to the start_row

    # Set the page load timeout in the driver
    driver.set_page_load_timeout(timeout)

    # Counter for tracking crawled URLs
    crawled_count = 0

    # Open the output CSV for writing
    with open(output_csv_path, 'a', newline='', encoding='utf-8') as output_file:
        csv_writer = csv.writer(output_file)
        
        # Write headers if the file is empty or just append if it already exists
        if not os.path.exists(output_csv_path) or start_row == 0:
            csv_writer.writerow(['Domain', 'Content', 'Label'])

        # Iterate through each row (domain) starting from start_row
        for index, row in df.iterrows():
            domain = row['Domain']
            label = row['Label']

            try:
                # Check if the domain already contains 'http' or 'https', else add 'http://'
                if not domain.startswith(('http://', 'https://')):
                    url = f"http://{domain}"
                else:
                    url = domain

                # Increment the counter and print the progress
                crawled_count += 1

                print(f"{crawled_count} - Fetching {url} ...")
                driver.get(url)

                # Extract the text content from the webpage body
                page_text = driver.find_element(By.TAG_NAME, "body").text

                # Write the domain, page text, and label to the output CSV file
                csv_writer.writerow([domain, page_text, label])
                print(f"Saved content for {domain}")

            except TimeoutException:
                print(f"Failed to fetch {domain}: Timeout after {timeout} seconds.")
                # Write an empty string for content if fetching fails
                csv_writer.writerow([domain, '', label])

            except Exception as e:
                print(f"Failed to fetch {domain}: {e}")
                # Write an empty string for content if fetching fails
                csv_writer.writerow([domain, '', label])

            # Pause to avoid overloading the server (optional)
            time.sleep(1)

    print(f"Completed fetching web content from row {start_row}. Total URLs crawled: {crawled_count}. Saved to {output_csv_path}")








In [2]:
# Define Chrome options
chrome_options = Options()
# chrome_options.add_argument("--headless")
# chrome_options.add_argument("--no-sandbox")
# Add more options here if needed

# Define paths
user_home_dir = os.path.expanduser("~")
chrome_binary_path = os.path.join(user_home_dir, "chrome-linux64", "chrome")
chromedriver_path = os.path.join(user_home_dir, "chromedriver-linux64", "chromedriver")

# Set binary location and service
chrome_options.binary_location = chrome_binary_path
webdriver_service = Service(chromedriver_path)

driver = webdriver.Chrome(service=webdriver_service, options=chrome_options)

In [1]:
import pandas as pd

# Replace with the path to your domainhanan.csv file
csv_file_path = '/mnt/f/SKRIPSI/Data/PhishStorm/urlset_and_content.csv'

# Load the CSV file into a DataFrame
df = pd.read_csv(csv_file_path)

# Print the first 5 rows of the DataFrame (head)
print(df.head())

                                Domain  \
0              www.sec.gov/edgar.shtml   
1  www.taxsites.com/associations2.html   
2        www.crescan.com/pakistanicma/   
3      www.ficpa.org/content/home.aspx   
4             pacioli.loyola.edu/aecm/   

                                             Content  Label  
0  Home | Jobs | Fast Answers | Site Map | Search...    0.0  
1  Server Error\n404 - File or directory not foun...    0.0  
2  No Results Found\nThe page you requested could...    0.0  
3                                                NaN    0.0  
4                                                NaN    0.0  


In [2]:
# Print the number of rows in the DataFrame
print(f"Number of rows in df: {len(df)}")

Number of rows in df: 322


In [4]:
# Input and output file paths
input_csv_path = '/mnt/f/SKRIPSI/Data/PhishStorm/modified_urlset.csv'  # Update the correct path to your input CSV file
output_csv_path = '/mnt/f/SKRIPSI/Data/PhishStorm/urlset_and_content.csv'  # Update the correct path to save the output CSV file

In [5]:
start_row = 47902  # Start fetching from row 47904

# Ensure that you have a valid Selenium WebDriver instance
fetch_and_save_web_content_from_csv(input_csv_path, output_csv_path, driver, start_row)

Fetching http://www.sec.gov/edgar.shtml ...
Saved content for www.sec.gov/edgar.shtml
Fetching http://www.taxsites.com/associations2.html ...
Saved content for www.taxsites.com/associations2.html
Fetching http://www.crescan.com/pakistanicma/ ...
Saved content for www.crescan.com/pakistanicma/
Fetching http://www.ficpa.org/content/home.aspx ...
Failed to fetch www.ficpa.org/content/home.aspx: Timeout after 8 seconds.
Fetching http://pacioli.loyola.edu/aecm/ ...
Failed to fetch pacioli.loyola.edu/aecm/: Message: unknown error: net::ERR_NAME_NOT_RESOLVED
  (Session info: chrome=129.0.6668.58)
Stacktrace:
#0 0x559e5d47b13a <unknown>
#1 0x559e5d1615e0 <unknown>
#2 0x559e5d1597eb <unknown>
#3 0x559e5d149a09 <unknown>
#4 0x559e5d14b744 <unknown>
#5 0x559e5d149cbd <unknown>
#6 0x559e5d14952f <unknown>
#7 0x559e5d149422 <unknown>
#8 0x559e5d147493 <unknown>
#9 0x559e5d147ada <unknown>
#10 0x559e5d163f41 <unknown>
#11 0x559e5d1f58c5 <unknown>
#12 0x559e5d1d5b22 <unknown>
#13 0x559e5d1f4d7d <unkn

KeyboardInterrupt: 

In [6]:
# Input and output file paths
input_csv_path = '/home/jordinia/Projects/ML-Malicious-Web-Scraper/domainhanan.csv'  # Update the correct path to your input CSV file
output_csv_path = '/home/jordinia/Projects/ML-Malicious-Web-Scraper/domainhanan_and_content2.csv'

In [7]:
# Open the input CSV file and read the domains
with open(input_csv_path, 'r', encoding='utf-8') as csv_file:
    csv_reader = csv.reader(csv_file)
    domains = [row[0] for row in csv_reader]  # Assuming the domain is in the first column

In [8]:
print(domains[60])

https://www.un.org/en/


In [9]:
fetch_and_save_web_content(domains, output_csv_path, driver)

print(f"Data saved to {output_csv_path}")

Fetching Domain ...
Failed to fetch Domain: Message: invalid argument
  (Session info: chrome=129.0.6668.58)
Stacktrace:
#0 0x56089a76213a <unknown>
#1 0x56089a44843d <unknown>
#2 0x56089a42fda7 <unknown>
#3 0x56089a42e3e8 <unknown>
#4 0x56089a42eada <unknown>
#5 0x56089a44af41 <unknown>
#6 0x56089a4dc8c5 <unknown>
#7 0x56089a4bcb22 <unknown>
#8 0x56089a4dbd7d <unknown>
#9 0x56089a4bc8c3 <unknown>
#10 0x56089a48a6b3 <unknown>
#11 0x56089a48b68e <unknown>
#12 0x56089a72cb3b <unknown>
#13 0x56089a730ac1 <unknown>
#14 0x56089a719335 <unknown>
#15 0x56089a731642 <unknown>
#16 0x56089a6fe49f <unknown>
#17 0x56089a751038 <unknown>
#18 0x56089a751203 <unknown>
#19 0x56089a760f8c <unknown>
#20 0x7fa9e46d0ac3 <unknown>

Fetching https://www.kompas.com/ ...
Saved content for https://www.kompas.com/
Fetching https://www.liputan6.com/ ...


KeyboardInterrupt: 

In [5]:
# Example of fetching a URL
driver.get("https://smkn4cilegon.sch.id/cilok/?gelar=789bet")

In [6]:
page_text = driver.find_element(By.TAG_NAME, "body").text
print(f"Full page text: {page_text}")  # Print the first 500 characters of the page text


Full page text: 789BET SLOT GACOR
SLOT GACOR MAXWIN SLOT GACOR HARI INI
SLOT ONLINE
LOGIN DAFTAR
Cari
Kategori
LazMall
Pulsa & Tagihan
Voucher & Diskon
LazBlog
789BET : Sejarah Berdirinya SMK Negeri 4 Cilegon Menjadi Contoh Sekolah Lain
6.482.132 Penilaian
Merek: 789BET
789BET Berdirinya Sekolah SMK Negeri 4 cilegon tidak lepas dari bantuan kementrian pendidikan untuk Provinsi Banten terkait sekolah Kemaritiman. Kota Cilegon menjadi prioritas karena berada di ujung Pulau Jawa yang menghubungkan pulau Jawa dengan Sumatera dan berada di daerah Pelabuhan dengan catatan siap menyiapkan lahannya sedangkan bangunannya dari pusat.
Rp.10.000
Rp.50.000-80%
Kuantitas
Login
DAFTAR SEKARANG
Dijual oleh SLOT GACOR


In [ ]:
html_content = driver.page_source
print(f"HTML content: {html_content[:500]}")  # Print first 500 characters of HTML


In [10]:



# # Example usage
# check_domain_availability("xnxx.com")
# check_domain_availability("pornhub.com")
# check_domain_availability("aseanenery.com")
# check_domain_availability("aseanenergy.org")